In [60]:
%%capture
import mrcnn
import mrcnn.config
import mrcnn.model
import mrcnn.visualize
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
%%capture captured
CLASS_NAMES = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

class SimpleConfig(mrcnn.config.Config):
    NAME = "coco_inference"
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    NUM_CLASSES = len(CLASS_NAMES)

model = mrcnn.model.MaskRCNN(mode="inference", 
                             config=SimpleConfig(),
                             model_dir=os.getcwd())

model.load_weights("/home/kabilan/Desktop/caption/dataset/mask_rcnn_coco.h5", by_name=True)

In [103]:
class Node:
    def __init__(self,start,end,class_id,class_name):
        self.start = start
        self.end = end
        self.class_id = class_id
        self.class_name = class_name
        
    def get_area(self):
        height = self.end[1] - self.start[1]
        width = self.end[0] - self.start[0]
        return width * height

In [104]:
def get_objects_and_locations(image_path):
    image = cv2.imread(image_path,1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rois = model.detect([image], verbose=0)[0]
    return rois

In [105]:
img_path = "/home/kabilan/Desktop/caption/dataset/images/10815824_2997e03d76.jpg"
img_path = "/home/kabilan/Desktop/caption/steps/step_6.jpeg"
image = cv2.imread(img_path,1)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
res = get_objects_and_locations(img_path)

In [106]:
im_width,im_height = image.shape[1],image.shape[0]

In [107]:
cur_pos = (im_height,int(im_width/2))

In [108]:
def get_angle(node_a,node_b):
    x1,y1 = node_a.end
    x2,y2 = node_b.end
    tan = (y2-y1)/(x2-x1)
    return tan

def distance(node_a,node_b):
    return np.linalg.norm(np.array(node_a.end) - np.array(node_b.end))

In [109]:
objects = []

In [110]:
for i in range(len(res['rois'])):
    class_name = CLASS_NAMES[res['class_ids'][i]]
    class_id = res['class_ids'][i]
    points = res['rois'][i]
    start = (points[1],points[0])
    end = (points[3],points[2])
    score = res['scores'][i]
    
    ob = Node(start,end,class_id,class_name)
    if score > 0.8:
        objects.append(ob)

In [111]:
pos_node = Node(cur_pos,cur_pos,-1,"current_position")

In [112]:
target_node = None
target_class_id = CLASS_NAMES.index('bottle') 

In [113]:
def parse_direction(pos_node,old_target_node,target_node):
    dir_str = ""
    angle = get_angle(pos_node,target_node)
    movement = ""
    if angle < -1.0:
        movement = "left"
    elif angle > 1.0:
        movement = "right"
    else:
        movement = "front"
    dis_str = ""
    if old_target_node is not None:
        if old_target_node.get_area() < target_node.get_area():
            dis_str = "closer"
        else:
            dis_str = "away"
    return movement , dis_str

In [118]:
for each in objects:
    if each.class_id == target_class_id:
        print(parse_direction(pos_node,target_node,each))
        target_node = each
        

('left', 'closer')
('left', 'away')


/home/kabilan/Desktop/caption/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in long_scalars
  after removing the cwd from sys.path.
